In [1]:
import numpy as np
import pandas as pd
from pathlib import Path
import os.path

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split

import tensorflow as tf

from sklearn.metrics import confusion_matrix, classification_report

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: l

In [24]:
train_image_dir = Path('../input/cnfood-241/train600x600')
val_image_dir = Path('../input/cnfood-241/val600x600')
test_image_dir = Path('../input/fooddataset/Food')

In [32]:
train_filepaths = list(train_image_dir.glob(r'**/*.jpg'))
train_labels = list(map(lambda x: os.path.split(os.path.split(x)[0])[1], train_filepaths))

train_filepaths = pd.Series(train_filepaths, name='Filepath').astype(str)
train_labels = pd.Series(train_labels, name='Label')

train_images = pd.concat([train_filepaths, train_labels], axis=1)

category_samples = []
for category in train_images['Label'].unique():
    category_slice = train_images.query("Label == @category")
    max_samples = min(125, category_slice.shape[0])
    category_samples.append(category_slice.sample(max_samples, random_state=1))
train_image_df = pd.concat(category_samples, axis=0).sample(frac=1.0, random_state=1).reset_index(drop=True)

In [33]:
train_image_df

,Filepath,Label
0,../input/cnfood-241/train600x600/127/000296.jpg,127
1,../input/cnfood-241/train600x600/099/000425.jpg,099
2,../input/cnfood-241/train600x600/095/000313.jpg,095
3,../input/cnfood-241/train600x600/072/000239.jpg,072
4,../input/cnfood-241/train600x600/078/000034.jpg,078
...,...,...
29879,../input/cnfood-241/train600x600/237/124.28209...,237
29880,../input/cnfood-241/train600x600/000/000219.jpg,000
29881,../input/cnfood-241/train600x600/075/000704.jpg,075
29882,../input/cnfood-241/train600x600/183/000539.jpg,183


In [34]:
val_filepaths = list(val_image_dir.glob(r'**/*.jpg'))
val_labels = list(map(lambda x: os.path.split(os.path.split(x)[0])[1], val_filepaths))

val_filepaths = pd.Series(val_filepaths, name='Filepath').astype(str)
val_labels = pd.Series(val_labels, name='Label')

val_images = pd.concat([val_filepaths, val_labels], axis=1)

category_samples = []
for category in val_images['Label'].unique():
    category_slice = val_images.query("Label == @category")
    max_samples = min(50, category_slice.shape[0])
    category_samples.append(category_slice.sample(max_samples, random_state=1))
val_image_df = pd.concat(category_samples, axis=0).sample(frac=1.0, random_state=1).reset_index(drop=True)

In [35]:
val_image_df

,Filepath,Label
0,../input/cnfood-241/val600x600/136/000008.jpg,136
1,../input/cnfood-241/val600x600/125/000097.jpg,125
2,../input/cnfood-241/val600x600/112/000018.jpg,112
3,../input/cnfood-241/val600x600/014/000035.jpg,014
4,../input/cnfood-241/val600x600/014/000050.jpg,014
...,...,...
10322,../input/cnfood-241/val600x600/071/000047.jpg,071
10323,../input/cnfood-241/val600x600/063/000062.jpg,063
10324,../input/cnfood-241/val600x600/016/000130.jpg,016
10325,../input/cnfood-241/val600x600/108/000045.jpg,108


In [36]:
test_filepaths = list(test_image_dir.glob(r'**/*.jpg*'))
test_filepaths.extend(list(test_image_dir.glob(r'**/*.png*')))
test_filepaths = pd.Series(test_filepaths, name='Filepath').astype(str)
test_image_df = pd.concat([test_filepaths], axis=1)

In [37]:
test_image_df

,Filepath
0,../input/fooddataset/Food/20170521_091328.jpg
1,../input/fooddataset/Food/IMG_5028.jpg
2,../input/fooddataset/Food/IMG_9594.jpg
3,../input/fooddataset/Food/IMG_7895.jpg
4,../input/fooddataset/Food/marine iguana huntin...
...,...
663,../input/fooddataset/Food/IMG_7004.png
664,../input/fooddataset/Food/IMG_7003.png
665,../input/fooddataset/Food/IMG_5940.png
666,../input/fooddataset/Food/IMG_7007.png


In [39]:
train_image_df['Label'].value_counts()
val_image_df['Label'].value_counts()


136    50
188    50
068    50
013    50
119    50
       ..
211    13
109    12
186    11
213    11
169     4
Name: Label, Length: 241, dtype: int64

In [41]:
train_generator = tf.keras.preprocessing.image.ImageDataGenerator(
    preprocessing_function=tf.keras.applications.mobilenet_v2.preprocess_input
)

val_generator = tf.keras.preprocessing.image.ImageDataGenerator(
    preprocessing_function=tf.keras.applications.mobilenet_v2.preprocess_input
)

test_generator = tf.keras.preprocessing.image.ImageDataGenerator(
    preprocessing_function=tf.keras.applications.mobilenet_v2.preprocess_input
)

In [42]:
train_images = train_generator.flow_from_dataframe(
    dataframe=train_image_df,
    x_col='Filepath',
    y_col='Label',
    target_size=(224, 224),
    color_mode='rgb',
    class_mode='categorical',
    batch_size=32,
    shuffle=True,
    seed=42
)

val_images = val_generator.flow_from_dataframe(
    dataframe=val_image_df,
    x_col='Filepath',
    y_col='Label',
    target_size=(224, 224),
    color_mode='rgb',
    class_mode='categorical',
    batch_size=32,
    shuffle=True,
    seed=42
)

test_images = test_generator.flow_from_dataframe(
    dataframe=test_image_df,
    x_col='Filepath',
    y_col=None,
    target_size=(224, 224),
    color_mode='rgb',
    class_mode=None,
    batch_size=32,
    shuffle=False,
    validate_filenames=False
)

Found 29884 validated image filenames belonging to 241 classes.
Found 10327 validated image filenames belonging to 241 classes.
Found 668 non-validated image filenames.


In [43]:
pretrained_model = tf.keras.applications.MobileNetV2(
    input_shape=(224, 224, 3),
    include_top=False,
    weights='imagenet',
    pooling='avg'
)

pretrained_model.trainable = False

9406464/9406464 [==============================] - 0s 0us/step


In [48]:
inputs = pretrained_model.input

x = tf.keras.layers.Dense(128, activation='relu')(pretrained_model.output)
x = tf.keras.layers.Dense(128, activation='relu')(x)

outputs = tf.keras.layers.Dense(241, activation='softmax')(x)

model = tf.keras.Model(inputs, outputs)


print(model.summary())

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 Conv1 (Conv2D)                 (None, 112, 112, 32  864         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 bn_Conv1 (BatchNormalization)  (None, 112, 112, 32  128         ['Conv1[0][0]']                  
                                )                                                           

In [47]:
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

history = model.fit(
    train_images,
    validation_data=val_images,
    epochs=100,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(
            monitor='val_loss',
            patience=3,
            restore_best_weights=True
        )
    ]
)

Epoch 1/100


InvalidArgumentError: Graph execution error:

Detected at node 'categorical_crossentropy/softmax_cross_entropy_with_logits' defined at (most recent call last):
    File "/opt/conda/lib/python3.10/runpy.py", line 196, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "/opt/conda/lib/python3.10/runpy.py", line 86, in _run_code
      exec(code, run_globals)
    File "/opt/conda/lib/python3.10/site-packages/ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "/opt/conda/lib/python3.10/site-packages/traitlets/config/application.py", line 1043, in launch_instance
      app.start()
    File "/opt/conda/lib/python3.10/site-packages/ipykernel/kernelapp.py", line 728, in start
      self.io_loop.start()
    File "/opt/conda/lib/python3.10/site-packages/tornado/platform/asyncio.py", line 195, in start
      self.asyncio_loop.run_forever()
    File "/opt/conda/lib/python3.10/asyncio/base_events.py", line 603, in run_forever
      sel

In [ ]:
predictions = np.argmax(model.predict(test_images), axis=1)

In [ ]:
predictions